In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import json
import tensorflow as tf
import numpy as np

from mrcnn import visualize
import mrcnn.model as modellib
from mrcnn.model import log
import perception

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
config = perception.CocoConfig()
class_names = ['BG', 'person', 'rider', 'car', 'bus', 'truck', 'bike', 'motor', 'traffic light', 'traffic sign', 'train']

In [4]:
image_dir = "/home/dtilak/object_detection/test_images/input/"
output_image_path = "/home/dtilak/object_detection/test_images/output_rgb_mask_rcnn_resnet101_fpn_data_3/"
weights_path = "/home/dtilak/object_detection/training_log_rgb/coco20190811T1837/mask_rcnn_coco_0005.h5"
MODEL_DIR = "/home/dtilak/object_detection/training_log_rgb/"

In [5]:
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_weights(weights_path, by_name=True)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Re-starting from epoch 5


In [6]:
def get_image_list(image_dir):
    files= os.listdir(image_dir) 
    s = []
    for file in files: 
        str_name = file[:21]
        s.append(str_name) 
    return s

image_list = get_image_list(image_dir)

In [7]:
def get_individual_class_details(class_ids, scores, rois):
    
    result_array = dict()
    for index in range(len(class_ids)):

        predicted_class = class_names[class_ids[index]]
        predicted_class_score = scores[index].tolist()
        predicted_class_boxes = rois[index].tolist()

        result_array[predicted_class] = ({"Confidence":[predicted_class_score]}, 
                                                 {"Bounding box":[predicted_class_boxes]})

    #print(result_array)
    return result_array

In [ ]:
test_annos = dict()
JSON_PATH_OUTPUT = os.path.join(output_image_path, "results.json")
for i in range(len(image_list)):
    image_path = os.path.join(image_dir,image_list[i]) 
    print(image_path)
    image = cv2.imread(image_path)

    height, width = image.shape[:2]

    results = model.detect([image], verbose=1)


    r = results[0]
    image_ir = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], 
                                title="Predictions")
    plt.savefig(os.path.join(output_image_path,image_list[i]), bbox_inches='tight')
     
    class_details = get_individual_class_details(r['class_ids'], r['scores'], r['rois'])
    test_annos[i] = ({image_list[i] : class_details})
    
    #test_annos[image_list[i]] = (get_individual_class_details(r['class_ids'], r['scores'], r['rois']))
        
    if i==2:
        break
        
test_annos = {"Mask R-CNN with ResNet101 backbone and Feature Pyramid Network (FPN)": test_annos}
#print(test_annos)
fd = open(JSON_PATH_OUTPUT, 'w')
json.dump(test_annos,fd,indent=4,sort_keys=True)

fd.close()